In [4]:
import requests
import pandas as pd
import json
from scripts.env import HEADERS
from scripts.fravega import get_product_data, get_product_attributes
import os

In [5]:
productos = os.listdir('datos/lenovo/productos')

In [6]:
dataframes = []
for p in productos:
    with open(f'datos/lenovo/productos/{p}', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    dataframes.append(pd.DataFrame([get_product_attributes(data)]))

In [7]:
df_atributos = pd.concat(dataframes)

In [8]:
df_productos = pd.read_csv('datos/lenovo/productos-lenovo-fravega.csv')
df_productos.head()

,id,title,katalogCategoryId,slug,brand_id,brand_name,sku_id,categories,list_price,sale_price
0,686697d1a9c2fb5f11739b89,Lenovo LOQ Intel I5 ssd512gb 12gb FHD 15.6 rt...,5a301c031400008a0049181b,-lenovo-loq-intel-i5-ssd512gb-12gb-fhd-15-6-rt...,5a301bdb14000072004905f2,Lenovo,22506825,"['Notebooks', 'Informática']",3890000.00,3290000.00
1,60dc6f0c1000007980d9eb23,Notebook Lenovo Gamer Core i7 1165G7 12gb Ram...,5a301c031400008a0049181b,-notebook-lenovo-gamer-core-i7-1165g7-12gb-ram...,5a301bdb14000072004905f2,Lenovo,20005932,"['Notebooks', 'Informática']",2125200.00,1383113.00
2,68a8aae1b8267b7b96c7f446,Notebook Lenovo Slim Procesador Intel Core I7...,5a301c031400008a0049181b,-notebook-lenovo-slim-procesador-intel-core-i7...,5a301bdb14000072004905f2,Lenovo,22581762,"['Notebooks', 'Informática']",1890000.00,1574999.00
3,67e698b36684324c88af99f2,2020 Lenovo IdeaPad 3 15.6&quot; Portátil Inte...,5a301c0a1400007200491a8b,2020-lenovo-ideapad-3-15-6&quot-portatil-intel...,5a301bdb14000072004905f2,Lenovo,22459077,"['Base para Netbooks y Notebooks', 'Accesorios...",1187357.00,1009253.45
4,684aecb9d2d4f1fc2fb0606b,64GB LN DDR5-4800 MHZ RDIMM IN,5a301c0e1400008700491bc5,64gb-ln-ddr5-4800-mhz-rdimm-in,5a301bdb14000072004905f2,Lenovo,990233415,"['Memorias', 'Informática']",1402123.75,1121699.00


In [9]:
df_atributos.head()

,product_id,product_specifications,product_attributes
0,60dc6f0c1000009080d9eb22,"[{'Procesador': 'Intel Core i7-1165G7'}, {'Mod...",[{'Descripción': '<p>Rendimiento sólido y prac...
0,60dc6f0c1000007980d9eb23,"[{'Procesador': 'Intel Core i7-1165G7'}, {'Mod...",[{'Descripción': '<p>Rendimiento sólido y prac...
0,610aac921300005101ed35c6,"[{'Procesador': 'Intel Core i7-1165G7'}, {'Mod...","[{'Notebook Gamer': False}, {'Tipo de procesad..."
0,6410c3dd3600006070b47c2d,[],[]
0,6445740c370000b4146a81b9,"[{'Micrófono': True}, {'Tipo de uso': 'In Ear'...","[{'Micrófono': True}, {'Tipo de uso': 'In Ear'..."


In [10]:
full = pd.merge(

    df_productos,
    df_atributos,
    left_on='id',
    right_on='product_id',
    how='outer'
)


In [11]:
full = full.dropna().sort_values('title').reset_index(drop=True).drop(columns=['product_id'])

In [12]:
full

,id,title,katalogCategoryId,slug,brand_id,brand_name,sku_id,categories,list_price,sale_price,product_specifications,product_attributes
0,686697d1a9c2fb5f11739b89,Lenovo LOQ Intel I5 ssd512gb 12gb FHD 15.6 rt...,5a301c031400008a0049181b,-lenovo-loq-intel-i5-ssd512gb-12gb-fhd-15-6-rt...,5a301bdb14000072004905f2,Lenovo,22506825,"['Notebooks', 'Informática']",3890000.00,3290000.00,"[{'Procesador': 'Intel I5'}, {'Mod. procesador...","[{'Notebook Gamer': True}, {'Tipo de procesado..."
1,60dc6f0c1000007980d9eb23,Notebook Lenovo Gamer Core i7 1165G7 12gb Ram...,5a301c031400008a0049181b,-notebook-lenovo-gamer-core-i7-1165g7-12gb-ram...,5a301bdb14000072004905f2,Lenovo,20005932,"['Notebooks', 'Informática']",2125200.00,1383113.00,"[{'Procesador': 'Intel Core i7-1165G7'}, {'Mod...",[{'Descripción': '<p>Rendimiento sólido y prac...
2,68a8aae1b8267b7b96c7f446,Notebook Lenovo Slim Procesador Intel Core I7...,5a301c031400008a0049181b,-notebook-lenovo-slim-procesador-intel-core-i7...,5a301bdb14000072004905f2,Lenovo,22581762,"['Notebooks', 'Informática']",1890000.00,1574999.00,[{'Procesador': 'Procesador Intel Core i7 1362...,"[{'Compra Internacional': False}, {'Notebook G..."
3,67e698b36684324c88af99f2,2020 Lenovo IdeaPad 3 15.6&quot; Portátil Inte...,5a301c0a1400007200491a8b,2020-lenovo-ideapad-3-15-6&quot-portatil-intel...,5a301bdb14000072004905f2,Lenovo,22459077,"['Base para Netbooks y Notebooks', 'Accesorios...",1187357.00,1009253.45,[{'Origen': 'United States'}],"[{'Compra Internacional': True}, {'Origen': 'U..."
4,684aecb9d2d4f1fc2fb0606b,64GB LN DDR5-4800 MHZ RDIMM IN,5a301c0e1400008700491bc5,64gb-ln-ddr5-4800-mhz-rdimm-in,5a301bdb14000072004905f2,Lenovo,990233415,"['Memorias', 'Informática']",1402123.75,1121699.00,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
495,681b85e2e065019ae7e76a8c,"Tableta Lenovo Tab K10 ZA8S0000US de 10,3 pulg...",5a301c041400008a004918a5,tableta-lenovo-tab-k10-za8s0000us-de-10-3-pulg...,5a301bdb14000072004905f2,Lenovo,22481757,"['Tablets', 'Tablets', 'Informática']",490762.00,417147.70,"[{'Tamaño de Pantalla': 10.3}, {'Memoria Inter...","[{'Compra Internacional': True}, {'Tipo de USB..."
496,67784e102b00006ad69e3187,"Tablets Lenovo Tab K11 MTK 8GB 128GB SSD 11"" +...",5a301c041400008a004918a5,tablets-lenovo-tab-k11-mtk-8gb-128gb-ssd-11-te...,5a301bdb14000072004905f2,Lenovo,22060207,"['Tablets', 'Tablets', 'Informática']",788999.00,559998.00,"[{'Wi-Fi': 'Sí'}, {'GPS': False}, {'NFC': Fals...","[{'Compra Internacional': False}, {'Tipo de US..."
497,68af1f21968d9aff922957e5,Tarjeta de memoria microSD Kingston Canvas Go ...,5a301c0c1400007700491b23,tarjeta-de-memoria-microsd-kingston-canvas-go-...,5a301bdb14000072004905f2,Lenovo,990246993,"['Tarjetas de memoria', 'Almacenamiento', 'Inf...",136998.75,109599.01,[],[]
498,681919b4ed4494c28e66ac00,"ThinkPad T16 3ra Gen Intel Ultra 7 32GB 512GB 16""",5a301c031400008a0049181b,thinkpad-t16-3ra-gen-intel-ultra-7-32gb-512gb-16-,5a301bdb14000072004905f2,Lenovo,22479069,"['Notebooks', 'Informática']",3554999.00,2899998.00,"[{'Pantalla táctil': False}, {'Wi-Fi': False},...","[{'Compra Internacional': False}, {'Notebook G..."


In [13]:
full.to_csv('datos/lenovo/productos-lenovo.csv', index=False)